# Lab 1: Configuração do Banco de Dados (PostgreSQL)

Este notebook executa a configuração do banco de dados PostgreSQL de forma interativa. As células de código a seguir irão se conectar ao contêiner do banco de dados e executar os scripts de criação de tabelas (DDL) e inserção de dados (DML) usando Python.

## Inicializar os bancos de dados

Etapa obrigatória, pois todos os serviços iniciam desligados para que o MYBinder consiga inicializar o Container.

In [ ]:
!/usr/local/bin/entrypoint.sh

## 1. Conexão e Execução dos Scripts

A célula abaixo contém todo o processo:
1.  Importa as bibliotecas necessárias.
2.  Define as queries DDL e DML.
3.  Estabelece uma conexão com o contêiner `petshop_db` (o nome do serviço no `docker-compose`).
4.  Cria um cursor e executa as queries.
5.  Confirma as transações e fecha a conexão.

In [ ]:
import psycopg2
import os

# As credenciais e o host são baseados no arquivo docker-compose.txt
DB_HOST = "localhost" # Nome do serviço no Docker Compose
DB_NAME = "postgres"
DB_USER = "postgres"
DB_USER_PWD = "postgres"

DDL_SCRIPT = """
CREATE TABLE IF NOT EXISTS tutor (
    tutor_id SERIAL PRIMARY KEY,
    name VARCHAR(255) NOT NULL,
    email VARCHAR(255) UNIQUE NULLS NOT DISTINCT,
    phone VARCHAR(20) UNIQUE NULLS NOT DISTINCT
);

CREATE TABLE IF NOT EXISTS pet (
    pet_id SERIAL PRIMARY KEY,
    tutor_id INTEGER NOT NULL REFERENCES tutor(tutor_id),
    name VARCHAR(100) NOT NULL,
    image_path VARCHAR(100),
    birth_date DATE,
    ignore_recommendation BOOLEAN DEFAULT FALSE,
    
    -- classification columns
    species VARCHAR(50) NOT NULL,
    animal_type VARCHAR(50),
    fur_type VARCHAR(50)
);

CREATE TABLE IF NOT EXISTS product (
    product_id SERIAL PRIMARY KEY,
    product_name VARCHAR(255) NOT NULL,
    category VARCHAR(100),
    price NUMERIC(10, 2) NOT NULL
);

CREATE TABLE IF NOT EXISTS purchase (
    purchase_id SERIAL PRIMARY KEY,
    tutor_id INTEGER NOT NULL REFERENCES tutor(tutor_id),
    product_id INTEGER NOT NULL REFERENCES product(product_id),
    purchase_date DATE NOT NULL,
    quantity INTEGER NOT NULL
);

-- Vaccination tables
CREATE TABLE IF NOT EXISTS vaccine_reference (
    vaccine_reference_id SERIAL PRIMARY KEY,
    vaccine_name VARCHAR(150) NOT NULL UNIQUE,
    description TEXT NOT NULL,
    target_species VARCHAR(50) NOT NULL, -- 'Cão', 'Gato' ou 'Ambos'
    equivalent_vaccines TEXT,-- Armazena nomes de vacinas equivalentes como uma string separada por ponto e vírgula
    mandatory BOOLEAN NOT NULL DEFAULT FALSE,
    first_dose_age_months FLOAT NOT NULL,
    booster_interval_months FLOAT
);

CREATE TABLE IF NOT EXISTS vaccination_record (
    vaccination_record_id SERIAL PRIMARY KEY,
    pet_id INTEGER NOT NULL REFERENCES pet(pet_id),
    vaccine_reference_id INTEGER NOT NULL REFERENCES vaccine_reference(vaccine_reference_id),
    application_date DATE NOT NULL,
    vaccine_batch VARCHAR(100),
    responsible_vet VARCHAR(255)
);


CREATE TABLE IF NOT EXISTS vaccine_recommendation (
    vaccine_recommendation_id SERIAL,
    pet_id INTEGER REFERENCES pet(pet_id),
    vaccine_name VARCHAR(255),
    description TEXT,
    mandatory BOOLEAN,
    suggested_date DATE,
    ignore_recommendation BOOLEAN DEFAULT FALSE,
    PRIMARY KEY (vaccine_recommendation_id, pet_id, vaccine_name)
);

-- Booking tables
CREATE TABLE IF NOT EXISTS booking (
    booking_id SERIAL PRIMARY KEY,
    pet_id INTEGER NOT NULL REFERENCES pet(pet_id),
    service_type VARCHAR(100) NOT NULL,
    booking_date TIMESTAMP WITH TIME ZONE NOT NULL,
    status VARCHAR(50) NOT NULL
);

CREATE TABLE IF NOT EXISTS booking_reference (
    booking_reference_id SERIAL,
    frequency_days INTEGER NOT NULL,

    -- classification columns
    species VARCHAR(50) NOT NULL,
    animal_type VARCHAR(50),
    fur_type VARCHAR(50),

    PRIMARY KEY (booking_reference_id, species, animal_type, fur_type)
);

CREATE TABLE IF NOT EXISTS booking_recommendation (
    booking_recommendation_id SERIAL,
    pet_id INTEGER REFERENCES pet(pet_id),
    suggested_date DATE,
    average_frequency_days INTEGER,
    ignore_recommendation BOOLEAN DEFAULT FALSE,
    PRIMARY KEY (booking_recommendation_id, pet_id)
);

-- LTV by Pet Profile table
CREATE TABLE IF NOT EXISTS ltv_by_pet_profile (
    ltv_by_pet_profile_id SERIAL,
    pet_profile VARCHAR(255),
    total_value DECIMAL(10, 2),
    PRIMARY KEY (ltv_by_pet_profile_id, pet_profile)
);

CREATE TABLE IF NOT EXISTS execution_history (
    execution_id SERIAL PRIMARY KEY,
    target_table VARCHAR(255) NOT NULL,
    start_time TIMESTAMP WITH TIME ZONE NOT NULL,
    end_time TIMESTAMP WITH TIME ZONE,
    status VARCHAR(50) NOT NULL, -- e.g., 'RUNNING', 'COMPLETED', 'FAILED',
    error_message TEXT,
    records_processed INTEGER
);
"""

DML_SCRIPT = """
INSERT INTO tutor (name, email, phone) VALUES
('Ana Carolina', 'ana.carolina@email.com', '55 91234-5678'),
('Bruno Martins', 'bruno.martins@email.com', '55 99876-5432')
ON CONFLICT (email) DO NOTHING;

INSERT INTO pet (tutor_id, name, species, animal_type, fur_type, birth_date) VALUES
(1, 'Bidu', 'Cão', 'Golden Retriever', 'Longo', NOW()),
(1, 'Luna', 'Gato', 'Siamês', 'Curto', NOW() - INTERVAL '1 year'),
(2, 'Thor', 'Cão', 'Shih Tzu', 'Longo', NOW() - INTERVAL '4 months')
ON CONFLICT DO NOTHING;

INSERT INTO booking (pet_id, service_type, booking_date, status) VALUES
(1, 'Banho e Tosa Completa', '2025-04-10 14:00:00', 'Realizado'),
(1, 'Banho e Tosa Completa', '2025-05-11 14:00:00', 'Realizado'),
(1, 'Banho e Tosa Completa', '2025-06-12 14:00:00', 'Realizado'),
(3, 'Banho', '2025-07-01 11:00:00', 'Realizado')
ON CONFLICT DO NOTHING;

INSERT INTO product (product_name, category, price) VALUES
('Ração para Cães de Pelo Longo', 'Alimentação', 75.50),
('Shampoo Hipoalergênico para Cães', 'Higiene', 30.00)
ON CONFLICT DO NOTHING;

INSERT INTO purchase (tutor_id, product_id, purchase_date, quantity) VALUES
(1, 1, '2025-06-15', 1),
(1, 2, '2025-06-15', 1),
(2, 1, '2025-06-22', 1)
ON CONFLICT DO NOTHING;

INSERT INTO vaccine_reference (vaccine_name, target_species, mandatory, description, equivalent_vaccines, first_dose_age_months, booster_interval_months) VALUES 
-- Aplicação Única 
('Aplicação de Microchip', 'Ambos', FALSE, 'Registro de aplicação de microchip de identificação.', NULL, 2, NULL),
 
-- Vacinas Não Essenciais para Ambos 
( 'Complexo Tosse dos Canis (Bordetella, Mucosa)', 'Ambos', FALSE, 'Vacinas vivas (intranasal ou oral) para proteção contra Bordetella bronchiseptica e/ou Parainfluenza.', NULL, 2, 12),
( 'Complexo Tosse dos Canis (Bordetella, Parenteral)', 'Ambos', FALSE, 'Vacina inativada (injetável) contra Bordetella bronchiseptica. Requer duas doses iniciais.', NULL, 2, 12),
( 'Borreliose de Lyme (Borrelia burgdorferi) canina', 'Ambos', FALSE, 'Recomendada para cães com alto risco de exposição a carrapatos em regiões onde a doença de Lyme é endêmica.', NULL, 2, 18),
( 'Gripe canina - Influenza Canina (H3N8 e H3N2)', 'Ambos', FALSE, 'Protege contra Bordetella bronchiseptica e/ou Parainfluenza. Considerar para cães em situações de risco, como canis, creches ou exposições.', NULL, 1.8, 12),
( 'Leishmaniose Canina', 'Ambos', FALSE, 'A vacinação é uma medida suplementar e não substitui o controle de flebotomíneos (vetores).', NULL, 1.8, 18), 

-- Vacinas Essenciais para Cães 
('V10 Canina (1 Dose) - Polivalente Canina Essencial', 'Cão', TRUE, 'Protege contra Cinomose, Parvovirose, Hepatite, Adenovírus, Parainfluenza, Coronavirose e 4 sorovares de Leptospirose.', 'V8 Canina', 1.45/* -- 45 dias / 31*/, NULL),  -- No WSAVA falase em 36 meses para cães de baixo risco, mas aqui mantemos 12 meses conforme solicitado. 
('V10 Canina (2 Dose) - Polivalente Canina Essencial', 'Cão', TRUE, 'Protege contra Cinomose, Parvovirose, Hepatite, Adenovírus, Parainfluenza, Coronavirose e 4 sorovares de Leptospirose.', 'V8 Canina', 2.29/* -- (45 dias + 26 dias) = 71 dias / 31 dias*/, NULL), -- No WSAVA falase em 36 meses para cães de baixo risco, mas aqui mantemos 12 meses conforme solicitado.
('V10 Canina (Dose Regular) - Polivalente Canina Essencial', 'Cão', TRUE, 'Protege contra Cinomose, Parvovirose, Hepatite, Adenovírus, Parainfluenza, Coronavirose e 4 sorovares de Leptospirose.', 'V8 Canina', 3.1/* -- (45 dias + 26 dias + 26 dias) = 97 dias / 31 dias*/, 12), -- No WSAVA falase em 36 meses para cães de baixo risco, mas aqui mantemos 12 meses conforme solicitado. 
('V8 Canina (1 Dose) - Polivalente Canina Essencial', 'Cão', TRUE, 'Protege contra Cinomose, Parvovirose, Hepatite, Adenovírus, Parainfluenza, Coronavirose e 2 sorovares de Leptospirose.', 'V10 Canina', 1.35/*-- 42 dias / 31 dias*/, NULL), -- No WSAVA falase em 36 meses para cães de baixo risco, mas aqui mantemos 12 meses conforme solicitado. 
('V8 Canina (2 Dose) - Polivalente Canina Essencial', 'Cão', TRUE, 'Protege contra Cinomose, Parvovirose, Hepatite, Adenovírus, Parainfluenza, Coronavirose e 2 sorovares de Leptospirose.', 'V10 Canina', 1.93/* -- (42 dias + 18 dias) / 31 dias*/, NULL), -- No WSAVA falase em 36 meses para cães de baixo risco, mas aqui mantemos 12 meses conforme solicitado. 
('V8 Canina (Dose Regular) - Polivalente Canina Essencial', 'Cão', TRUE, 'Protege contra Cinomose, Parvovirose, Hepatite, Adenovírus, Parainfluenza, Coronavirose e 2 sorovares de Leptospirose.', 'V10 Canina', 2.51/* -- (42 dias + 18 dias + 18 dias) / 31 dias*/, 12), -- No WSAVA falase em 36 meses para cães de baixo risco, mas aqui mantemos 12 meses conforme solicitado. 
( 'Raiva Canina - Antirrábica', 'Ambos', TRUE, 'Protege contra o vírus da Raiva. Obrigatória por lei no Brasil.', NULL, 3, 12),

-- Vacinas Não Essenciais para Cães 
('Giárdia Canina', 'Cão', FALSE, 'Protege contra Giardia lamblia.', NULL, 2, 18), -- Vacinas Essenciais para Gatos 
('V4 Felina (1 Dose) - Polivalente Essencial', 'Gato', TRUE, 'Protege contra Panleucopenia, Rinotraqueíte, Calicivirose e Clamidiose.', 'V5 Felina', 1.77, NULL),
('V4 Felina (2 Dose) - Polivalente Essencial', 'Gato', TRUE, 'Protege contra Panleucopenia, Rinotraqueíte, Calicivirose e Clamidiose.', 'V5 Felina', 2.74/* -- (55 dias + 30 dias) / 31 dias*/, NULL),
('V4 Felina (Dose Regular) - Polivalente Essencial', 'Gato', TRUE, 'Protege contra Panleucopenia, Rinotraqueíte, Calicivirose e Clamidiose.', 'V5 Felina', 3.7/* -- (55 dias + 30 dias + 30 dias) / 31 dias*/, 12),
('V5 Felina (1 Dose) - Polivalente Essencial', 'Gato', TRUE, 'Protege contra Panleucopenia, Rinotraqueíte, Calicivirose, Clamidiose e Leucemia Felina (FeLV).', 'V4 Felina', 2.25, NULL),
('V5 Felina (2 Dose) - Polivalente Essencial', 'Gato', TRUE, 'Protege contra Panleucopenia, Rinotraqueíte, Calicivirose, Clamidiose e Leucemia Felina (FeLV).', 'V4 Felina', 3.61/* -- (70 dias + 21) / 31 dias*/, NULL),
('V5 Felina (Dose Regular) - Polivalente Essencial', 'Gato', TRUE, 'Protege contra Panleucopenia, Rinotraqueíte, Calicivirose, Clamidiose e Leucemia Felina (FeLV).', 'V4 Felina', 1.8/* -- (70 dias + 21 + 21) / 31 dias*/, 12), 

-- Vacinas Não Essenciais para Gatos 
('Bordetella Bronchiseptica Felina (Intranasal)', 'Gato', FALSE, 'Não utilizada rotineiramente. Considerar para gatos em colônias muito grandes.', NULL, 1, 12)
ON CONFLICT DO NOTHING;
"""

try:
    conn = psycopg2.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_USER_PWD)
    cur = conn.cursor()
    
    print("Executando script DDL (criação de tabelas)...")
    cur.execute(DDL_SCRIPT)
    print("DDL executado com sucesso.")
    
    print("Executando script DML (inserção de dados)...")
    # Limpa as tabelas antes de inserir para garantir que o script seja idempotente
    cur.execute("""TRUNCATE TABLE
        execution_history,

        vaccination_record,
        vaccine_reference,
        vaccine_recommendation,

        booking,
        booking_reference,
        booking_recommendation,

        ltv_by_pet_profile,

        purchase,
        product,
        pet,
        tutor
        RESTART IDENTITY CASCADE;
    """)
    cur.execute(DML_SCRIPT)
    print("DML executado com sucesso.")
    
    conn.commit()
    print("Transação commitada.")
    
except psycopg2.OperationalError as e:
    print(f"Erro de conexão: {e}")
    print("Verifique se os contêineres Docker estão em execução ('docker-compose up -d') e se o nome do host do banco de dados ('{DB_HOST}') está correto.")
except Exception as e:
    print(f"Ocorreu um erro: {e}")
finally:
    if 'conn' in locals() and conn is not None:
        cur.close()
        conn.close()
        print("Conexão fechada.")

## 2. Verificando dados inseridos

Execute a célula abaixo para se conectar novamente e fazer uma consulta `SELECT` para verificar se os dados foram inseridos corretamente na tabela `pet`.

In [ ]:
try:
    conn = psycopg2.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_USER_PWD)
    cur = conn.cursor()
    
    cur.execute("SELECT * FROM pet;")
    rows = cur.fetchall()
    
    print("Registros encontrados:")
    for row in rows:
        print(row)
        
except Exception as e:
    print(f"Ocorreu um erro: {e}")
finally:
    if 'conn' in locals() and conn is not None:
        cur.close()
        conn.close()